# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube dans une collection MongoDB.

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
Toutes les questions sont à réalisés via des requêtes vers Mongo :
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [2]:
!pip install pymongo
import pandas as pd
import pymongo

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['exercices']
collection = database['youtube']

In [150]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [151]:
df_youtube_cat = pd.read_json("./data/US_category_id_v2.json", orient="values")

for new_col in ["channelId", "title", "assignable"]:
    df_youtube_cat.loc[new_col] = df_youtube_cat.loc["snippet"].apply(lambda x: x[new_col])

df_youtube_cat = df_youtube_cat.T.drop(columns=["snippet"])
df_youtube_cat.rename(columns={"id":"category_id"}, inplace=True)
df_youtube_cat.category_id = df_youtube_cat.category_id.apply(pd.to_numeric)

print(df_youtube_cat.info())
df_youtube_cat.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 6 columns):
kind           32 non-null object
etag           32 non-null object
category_id    32 non-null int64
channelId      32 non-null object
title          32 non-null object
assignable     32 non-null object
dtypes: int64(1), object(5)
memory usage: 1.8+ KB
None


,kind,etag,category_id,channelId,title,assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [152]:
df_youtube = df_youtube[df_youtube["views"] >= df_youtube["likes"] + df_youtube["dislikes"]]
df_youtube.drop(index=4867, inplace=True)
df_youtube.drop_duplicates(subset=["video_id"], inplace=True)
df_youtube.reset_index(drop=True, inplace=True)
df_youtube

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09
...,...,...,...,...,...,...,...,...,...,...,...
2358,BpGrq3LgbfA,Josie and The Pussycats ft. Cheryl Blossom | M...,BlackStardust,1,Riverdale - Josie and The Pussycats - CW - Che...,2205,11,0,0,https://i.ytimg.com/vi/BpGrq3LgbfA/default.jpg,22.10
2359,S9VIKOuZcds,My Sweet Jax (Tribute to a Cat),Hot Dad,23,cat|feline|pets|beloved|family member|grief|gr...,25037,2846,11,537,https://i.ytimg.com/vi/S9VIKOuZcds/default.jpg,22.10
2360,c74FjEqhCsc,Pistons LIVE Postgame 10.18.17: Stan Van Gundy,FOX SportsDetroit,17,[none],1280,11,0,0,https://i.ytimg.com/vi/c74FjEqhCsc/default.jpg,22.10
2361,a5NIg5yyHWo,Pawn Stars: An Original 1978 Superman Costume ...,HISTORY,24,history|history channel|history shows|history ...,400104,2432,123,386,https://i.ytimg.com/vi/a5NIg5yyHWo/default.jpg,22.10


### Importer les données

In [153]:
df = pd.merge(df_youtube, df_youtube_cat, on=["category_id"])

In [154]:
df.head()

,video_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,channelId,title_y,assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True


In [155]:
import numpy as np

def createDocsFromDF(df, collection=None, insertToDB=False):
    docs = [] 
    fields = [col for col in df.columns]
    for i in range(len(df)):
        doc = {col:df[col][i] for col in df.columns if col != 'index'}
        for key, val in doc.items():
            # we have to do this, because mongo does not recognize these np. types
            if type(val) == np.int64:
                doc[key] = int(val)
            if type(val) == np.float64:
                doc[key] = float(val)
            if type(val) == np.bool_:
                doc[key] = bool(val)
        docs.append(doc) 
    if insertToDB and collection:
        database.collection.insert_many(docs)
    return docs

createDocsFromDF(df, collection="collection", insertToDB=True)

[{'video_id': 'XpVt6Z1Gjjo',
  'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09,
  'kind': 'youtube#videoCategory',
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title_y': 'Entertainment',
  'assignable': True,
  '_id': ObjectId('605c547daf43ed4d2685255e')},
 {'video_id': '4MkC65emkG4',
  'titl

## Question 1  

In [156]:
df[df["channel_title"] == "Apple"]

,video_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,channelId,title_y,assignable
477,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
512,A9k88sMyiJM,Apple Watch Series 3 + Apple Music — Roll — Apple,Apple,28,Apple|airpods|apple watch|apple watch series 3...,782543,14647,1489,0,https://i.ytimg.com/vi/A9k88sMyiJM/default.jpg,18.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
559,REZl-ANYKKY,iPhone 8 Plus — Portraits of Her — Apple,Apple,28,Apple|new|iPhone|iPhone 8|iPhone 8 Plus|iOS 11...,117564,1240,459,0,https://i.ytimg.com/vi/REZl-ANYKKY/default.jpg,3.10,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True


In [157]:
for apple in database.collection.find({"channel_title":"Apple"}):
    print(apple)

{'_id': ObjectId('605c547daf43ed4d2685273b'), 'video_id': 'K4wEI5zhHB0', 'title_x': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 7860119, 'likes': 185853, 'dislikes': 26679, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 13.09, 'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"', 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title_y': 'Science & Technology', 'assignable': True}
{'_id': ObjectId('605c547daf43ed4d2685275e'), 'video_id': 'A9k88sMyiJM', 'title_x': 'Apple Watch Series 3 + Apple Music — Roll — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboa

## Question 2

In [158]:
df["category_id"].unique()

array([24, 28, 22, 23,  1, 26, 10, 25, 17, 27,  2, 20, 15, 29, 19, 43],
      dtype=int64)

In [159]:
print(len(database.collection.distinct("category_id")))

16


## Question 3

In [161]:
for x in database.collection.find():
#     new_tags = x["tags"].split("|")
#     database.collection.update_one({'video_id':x["video_id"]}, {"$set": {"tags": new_tags}})
    print(x["tags"])

['logan paul vlog', 'logan paul', 'logan', 'paul', 'olympics', 'logan paul youtube', 'vlog', 'daily', 'comedy', 'hollywood', 'parrot', 'maverick', 'bird', 'maverick clothes', 'diamond play button', 'logan paul diamond play button', '10M subscribers', 'logan paul 1 year vlogging', '1 year vlog', 'dwarf mamba play button', 'logan paul history', 'youtube history', '10M', '10M plaque', 'youtube button', 'diamond button', 'logang', 'logang 4 life']
['mtv', 'video', 'online', 'official', 'tv', 'television', 'watch', 'Hand in Hand', 'benefit', 'hurricane relief', 'money', 'hurricane harvey', 'hurricane irma', 'beyonce', 'george clooney', 'julia roberts', 'barbra stresiand', 'reese witherspoon', 'oprah winfrey', 'stars', 'celebrity', 'celebrities', 'charity', 'charities', 'aid', 'recovery', 'Houston', 'Florida', 'Miami', 'habitat for humanity', 'save the children', 'feeding texas', "mayor's fund for hurricane harvey relief", 'destruction', 'evacuation', 'evacuate', 'warning', 'storm', 'death',

['popular', 'video', 'videos', 'top', 'best', 'footage', 'clip', 'audio', 'film', 'films', 'youtube', 'amazing', 'viral', 'clips', 'world', 'news', 'golf', 'club', 'breaks', 'half', 'golfer', 'golfers', 'clubs', 'break', 'halfs', 'long', 'short', 'drive', 'champ', 'champion', 'champions', 'championship', 'championships', 'golfing', 'funny', 'wow', 'shocking', 'surprise']
['James Corden', 'The Late Late Show', 'Colbert', 'late night', 'late night show', 'Stephen Colbert', 'Comedy', 'monologue', 'comedian', 'impressions', 'celebrities', 'carpool', 'karaoke', 'CBS', 'Late Late Show', 'Corden', 'joke', 'jokes', 'funny', 'funny video', 'funny videos', 'humor', 'celebrity', 'celeb', 'hollywood', 'famous']
['michelle khare', 'buzzfeed michelle', 'fenty', 'beauty', 'review', 'makeup', 'mascara', 'waterproof makeup', 'THE ULTIMATE WATERPROOF DRUGSTORE MAKEUP TEST', 'eyeliner', 'kylie jenner', 'lip kit', 'fenty beauty', 'loreal', 'covergirl', 'extreme', 'try', 'for the first time', 'michelle', '

["My Girlfriend's Gay Friend", 'gay', 'friend', 'short film', 'crazy', 'twist', 'queer', 'homosexual', 'girlfriend', 'jelousy', 'shh', 'shush', 'rackquetteball', 'tennis', 'white', 'racquetball', 'squash', 'sixth sense', 'make out', 'beach', 'love', 'insane', 'offensive', 'horror', 'thriller', 'suspense', 'ghost', 'jealous', 'jealousy', 'Is it normal']
['Daisy Marquez', 'DaisyMarquez', 'Daisy Marques', 'Deisy Marquez', 'Deisy Marques', 'daisymarquez_', 'fall makeup tutorial', 'fall makeup look', 'fall makeup tutorial for brown eyes', 'fall makeup routine', 'orange fall makeup tutorial', 'orange fall makeup look', 'fall inspired makeup tutorial']
['itsgrace', 'funny', 'comedy', 'vlog', 'grace', 'helbig', 'gracehelbig', 'dailygrace', 'daily', 'tutorial', 'diy', 'lifestyle', 'audible', 'free trial', 'the gabbie show', 'adultolescence']
['Chain Snaps and Smashes through Windshield', 'Chain Snaps', 'Smashes', 'Windshield']
['ijustine', 'iphone 8', 'iphone 8 unboxing', 'apple watch series 3'

['Reese Witherspoon', 'Crate and Barrel', 'Christmas', 'Holidays', 'Entertaining', 'Prep', 'popcorn', 'cocktails']
['covergirl', 'new covergirl', 'the new covergirl', 'I am what I make Up', '#IAmWhatIMakeUp', 'katy perry', 'makeup', 'ayesha curry', 'issa rae', 'maye musk', 'massy arias', 'shelina moreda', 'women empowerment', 'women power', 'covergirl makeup', 'girl power', 'katy perry covergirl', 'covergirl commercial', 'female power', 'covergirl cosmetics', 'covergirl ad', 'ayeshacurry', 'katy perry makeup', 'cover girl models', 'power of women', 'power of a woman', 'women empowering women']
['ASMR', '[ASMR]', 'asmr', 'whisper', 'Maria', 'gentlewhispering', 'New', 'Latest', 'Sleep', 'Best', 'Sweet', 'Darling', 'Russian', 'relaxing', 'soothing', 'girl', 'lady', 'entertainment', 'comfort', 'gentle', 'tutorial', 'caring', 'meditation', 'nails', 'star', 'game', 'funny', 'sexy', 'calming', 'voice', 'tingles', 'chills', 'pleasant', 'insomnia', 'peace', 'love', 'hair', 'make', 'up', 'fashio

["Spinnin'", 'Records', "Spinnin' Records", 'Alok', 'Mathieu Koss', 'Big Jet Plane']
['andrew', 'huang', 'andrew huang', 'music', 'musician', 'producer', 'song', 'canadian', 'canada', 'toronto', 'ontario', 'Ableton live', 'instrument', 'music challenge', 'roomieofficial', 'rob scallon', 'boyinaband', 'test', 'quiz', 'trumpet', 'cello', 'clarinet', 'piano', 'nylon', 'guitar', 'acoustic', 'electric', 'roomie', 'dave brown', 'rob', 'scallon', 'dave', 'brown', 'david', 'boy in a band', 'challenge']
['St. Vincent', 'St Vincent', 'Annie Clark', 'Los Ageless', 'MASSEDUCTION', 'Masseduction', 'Willo Peron', 'Red Bull Music Academy', 'Red Bull', 'Los Angeles', 'Loma Vista Recordings', 'Loma Vista', 'Fear The Future.']
['Linkin Park', 'One More Light', 'Lyrics', 'Lyric Video', 'Official']
['Tom Petty', 'Tom Petty & The Heartbreakers', 'Rock n Roll Hall of Fame', 'American Girl', 'American Girl Live Performance', 'Tom Petty and the Heartbreakers', 'Rock Hall 2002 Inductions', '2002 Rock and Roll 

['Vanderbilt University']
['cgpgrey']
['survive in space', 'space', 'tardigrades', 'tardigrade', 'water bear', 'moss piglet', 'bacteria', 'cannibal', 'mass extinction', 'radiation', 'oxygen depletion', 'pressure', 'cryptobiosis', 'metabolic process', 'International Society of Tardigrade Hunters', 'how to find a tardigrade', 'how to see a tardigrade', 'weird science', 'Low Earth Orbit', 'life noggin', 'science', 'animation', 'animated science', 'blocko']
['Smelting', 'furnace', 'crucible', 'iron', 'ingot', 'minecraft', 'irl', 'magnetite', 'iron oxide', 'rust', 'coke', 'limestone', 'flux', 'induction', 'heater', 'panning', 'black sand', 'melting', 'cast iron', 'pig iron']
['john green', 'mental floss', 'library', 'book', 'george washington', 'dewey decimal', 'LoC', 'congress', 'university', 'public', 'school', 'banned', 'alexandria', 'egypt', 'ALA', 'american library association', 'bhuton', 'twitter', 'community bookshelf', 'kansas city', 'New York', 'tool', 'human', 'denmark', 'card', '

In [168]:
for x in database.collection.find({"tags": "[none]"}):
    database.collection.update_one({'tags': "[none]"}, {"$set":{"tags" : []}})
    print(x["tags"])

['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']
['[none]']

In [172]:
for x in database.collection.find():
    print(x["tags"])

['logan paul vlog', 'logan paul', 'logan', 'paul', 'olympics', 'logan paul youtube', 'vlog', 'daily', 'comedy', 'hollywood', 'parrot', 'maverick', 'bird', 'maverick clothes', 'diamond play button', 'logan paul diamond play button', '10M subscribers', 'logan paul 1 year vlogging', '1 year vlog', 'dwarf mamba play button', 'logan paul history', 'youtube history', '10M', '10M plaque', 'youtube button', 'diamond button', 'logang', 'logang 4 life']
['mtv', 'video', 'online', 'official', 'tv', 'television', 'watch', 'Hand in Hand', 'benefit', 'hurricane relief', 'money', 'hurricane harvey', 'hurricane irma', 'beyonce', 'george clooney', 'julia roberts', 'barbra stresiand', 'reese witherspoon', 'oprah winfrey', 'stars', 'celebrity', 'celebrities', 'charity', 'charities', 'aid', 'recovery', 'Houston', 'Florida', 'Miami', 'habitat for humanity', 'save the children', 'feeding texas', "mayor's fund for hurricane harvey relief", 'destruction', 'evacuation', 'evacuate', 'warning', 'storm', 'death',

['popular', 'video', 'videos', 'top', 'best', 'footage', 'clip', 'audio', 'film', 'films', 'youtube', 'amazing', 'viral', 'clips', 'world', 'news', 'golf', 'club', 'breaks', 'half', 'golfer', 'golfers', 'clubs', 'break', 'halfs', 'long', 'short', 'drive', 'champ', 'champion', 'champions', 'championship', 'championships', 'golfing', 'funny', 'wow', 'shocking', 'surprise']
['James Corden', 'The Late Late Show', 'Colbert', 'late night', 'late night show', 'Stephen Colbert', 'Comedy', 'monologue', 'comedian', 'impressions', 'celebrities', 'carpool', 'karaoke', 'CBS', 'Late Late Show', 'Corden', 'joke', 'jokes', 'funny', 'funny video', 'funny videos', 'humor', 'celebrity', 'celeb', 'hollywood', 'famous']
['michelle khare', 'buzzfeed michelle', 'fenty', 'beauty', 'review', 'makeup', 'mascara', 'waterproof makeup', 'THE ULTIMATE WATERPROOF DRUGSTORE MAKEUP TEST', 'eyeliner', 'kylie jenner', 'lip kit', 'fenty beauty', 'loreal', 'covergirl', 'extreme', 'try', 'for the first time', 'michelle', '

['Late Night', 'Seth Meyers', 'Kerry Washington', 'Eye', 'Scandal', 'Prada Bags', 'NBC', 'NBC TV', 'television', 'funny', 'talk show', 'comedy', 'humor', 'stand-up', 'parody', 'snl seth meyers', 'host', 'promo', 'seth', 'meyers', 'weekend update', 'news satire', 'satire', 'Django Unchained', 'Olivia Pope', 'Ray']
['malec', 'funny', 'humour', 'animation', 'français', 'japon', 'got', 'game of thrones', 'jon snow', 'anime', 'daenerys', 'saison 7', 'hbo', 'dessin anime']
['The Tonight Show', 'Jimmy Fallon', 'Lip Sync Battle', 'Miley Cyrus', 'NBC', 'NBC TV', 'Television', 'Funny', 'Talk Show', 'comedic', 'humor', 'snl', 'Fallon Stand-up', 'Fallon monologue', 'tonight', 'show', 'jokes', 'funny video', 'interview', 'variety', 'comedy sketches', 'talent', 'celebrities', 'video', 'clip', 'highlight', 'Younger Now', 'Hannah Montana', 'Wrecking Ball', 'Hole', 'Portugal', 'Feel It Still', 'Violet', 'Once In A Lifetime', 'Talking Heads', 'The Mans', 'rock', 'pop', 'alternative rock', 'music', 'lip 

['jeffree star', 'jeffree star approved', 'fenty beauty', 'fenty beauty review', 'fenty beauty rihanna', 'rihanna makeup line', 'rihanna makeup review', 'laura lee', 'manny mua', 'androgyny palette', 'sephora haul', 'get ready in my private jet', 'sephora rihanna review', 'fenty beauty hit or miss?', 'honest rihanna makeup review', 'drugstore makeup', 'patrick starrr', 'kkw beauty', 'kylie cosmetics', 'kylie jenner makeup', 'rihanna jeffree']
['fenty beauty', 'rihanna makeup', 'makeup for women of color', 'highlight', 'matchstick trio', 'WOC makeup', 'killawatt', 'trophy wife', 'bronzed makeup look', 'fall makeup', 'celebrity makeup', 'nyfw', 'new york fashion week']
['brothers green eats', 'brothers green', 'hurricane cooking', 'cooking with no power', 'easy recipes', 'pesto salami sandwich', 'cold pizza recipe', 'recipes', 'cooking', 'simple cooking', 'cooking without a stove', 'cooking without heat', 'irma', 'hurricane']
['Fenty Beauty', 'Rihanna makeup', 'WOC', 'makeup', 'Sponge', 

['LP', 'Death Valley', 'Other People', 'Lost On You', 'The McKittrick Hotel', 'Live']
['Keith Urban', 'Blake Shelton', 'Back To The Future', 'Little Jimmy Dickens', 'Bill Anderson', 'Opry', 'classic country', 'Country Music Hall Of Fame', 'John Fogerty', 'Mick Jaggar', 'Today', 'sony', 'vevo', 'cmt', 'arista', 'Franklin', 'Tennessee', 'TN', 'Kane Brown', 'CMA Awards', 'Carrie Underwood', 'class country', 'banjo', 'Arista Nashville', 'Brad Paisley', 'Country', 'Heaven South']
['Taylor Swift', 'Gorgeous', 'reputation']
['charlie puth', 'official video', 'how long charlie puth', 'how long has this been going on', 'how long - charlie puth', 'charlie puth - how long', 'how long lyrics', 'how long charlie', 'charlie how long', 'how long official', 'how long official video', 'how long music video', 'charlie puth how long', 'how long - charlie', 'atlantic records', 'charlie puth official', 'you just want attention', 'voicenotes charlie', 'voicenotes']
['Kelly Clarkson', 'Meaning of Life', 'MOL

['hennessey', 'hennessey performance', 'john hennessey', 'hennessy', 'tuner school', 'venom gt', 'venom f5', 'bugatti chiron', 'veyron', 'aston martin valkyrie', 'amg project one', 'sema', 'unveiling', 'event', 'top gear', 'top speed', '300 mph', 'mclaren', 'f1', 'p1', 'porsche 918', 'ferrari', 'laferrari', 'koenigsegg', 'agera rs', 'regera', 'lamborghini', 'pagani', 'zonda', 'huayra', 'hypercar', 'fastest', 'shell', 'pennzoil', '2017', 'guinness', 'world record', 'bp23', 'P15', 'vmax']
['IIHS', 'Insurance Institute', 'highway safety', 'Insurance Institute for Highway Safety', 'passenger-side', 'passenger side', 'small', 'overlap', 'front', 'frontal', 'offset', 'crash', 'test', 'rating', 'crashworthiness', 'evaluation', 'ratings', 'protection', 'new', 'program', 'midsize', 'cars']
[]
['PC', 'IGN', 'PS4', 'Doom', 'games', 'Trailer', 'Shooter', 'Xbox One', 'MachineGames', 'Id Software', 'ZeniMax Media', 'Bethesda Softworks', 'Wolfenstein 2: The New Colossus']
['gaming', 'games', 'psx', '

## Question 4

In [200]:
for x in database.collection.find({}).sort("views", -1).limit(5):
    print(x)

{'_id': ObjectId('605c547daf43ed4d26852b27'), 'video_id': 'tt2k8PGm-TI', 'title_x': 'ZAYN - Dusk Till Dawn ft. Sia', 'channel_title': 'ZaynVEVO', 'category_id': 10, 'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'], 'views': 36323498, 'likes': 1431683, 'dislikes': 28049, 'comment_total': 100661, 'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg', 'date': 13.09, 'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"', 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title_y': 'Music', 'assignable': True}
{'_id': ObjectId('605c547daf43ed4d26852b62'), 'video_id': 'MBdVXkSdhwU', 'title_x': "BTS (방탄소년단) 'DNA' Official MV", 'channel_title': 'ibighit', 'category_id': 10, 'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'], 'views': 20565649, 'likes': 1478119, 'dislikes': 46477, 'comment_total': 501435, 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg', 'date': 19.09, 'kind': 'youtube#videoCate

## Question 5

In [212]:
pipeline = [{
       "$group":
         {
           "_id": "$title_y",
           "avgByCategory": { "$avg": "$views" }
         }
     }]

res = database.collection.aggregate(pipeline)

for i in res:
    print(i)

{'_id': 'Travel & Events', 'avgByCategory': 300664.1111111111}
{'_id': 'Science & Technology', 'avgByCategory': 515854.13907284767}
{'_id': 'Education', 'avgByCategory': 368229.0425531915}
{'_id': 'Entertainment', 'avgByCategory': 555071.1740041929}
{'_id': 'Howto & Style', 'avgByCategory': 405748.2900763359}
{'_id': 'Comedy', 'avgByCategory': 828485.2676056338}
{'_id': 'Nonprofits & Activism', 'avgByCategory': 572023.2}
{'_id': 'People & Blogs', 'avgByCategory': 597891.1789883268}
{'_id': 'News & Politics', 'avgByCategory': 374883.0505050505}
{'_id': 'Music', 'avgByCategory': 699687.2095808383}
{'_id': 'Film & Animation', 'avgByCategory': 558289.1881188119}
{'_id': 'Shows', 'avgByCategory': 8259.0}
{'_id': 'Gaming', 'avgByCategory': 492390.2413793103}
{'_id': 'Autos & Vehicles', 'avgByCategory': 415129.0540540541}
{'_id': 'Sports', 'avgByCategory': 470686.1677852349}
{'_id': 'Pets & Animals', 'avgByCategory': 412381.6216216216}


## Question 6 

In [1]:
pipeline = [{
       "$group":
         {
           "_id": "$channel_title",
           "avgByLikes": { "$avg": "$likes" }
         }
     },
    {
        "$sort": {
            'avgByLikes': -1
        } 
    },
    {
        "$limit": 5
    }
    
]

res = database.collection.aggregate(pipeline)

for i in res:
    print(i)

NameError: name 'database' is not defined

[{'video_id': 'XpVt6Z1Gjjo',
  'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09,
  'kind': 'youtube#videoCategory',
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title_y': 'Entertainment',
  'assignable': True,
  '_id': ObjectId('605c40ecaf43ed4d2684af84')},
 {'video_id': '4MkC65emkG4',
  'titl